# **Fantasy Sports League Optimization**

## **Project Description**
This project focuses on optimizing team assignments in a fantasy sports league while ensuring a balanced distribution of talent and adherence to salary constraints. Given a dataset of players with attributes such as skill rating, cost, and position, we aim to assign them to five teams while following strict constraints and achieving a balanced league.

## **Constraints**
- Each team must consist of:
  - 1 Goalkeeper (GK)  
  - 2 Defenders (DEF)  
  - 2 Midfielders (MID)  
  - 2 Forwards (FWD)  
- Each player is assigned to exactly one team.  
- No team may exceed a total budget of **750 million €**.  
- The standard deviation of the average skill rating of all teams should be minimized to ensure fairness.  

## **Objective**
To generate a valid league configuration that follows all constraints and ensures that teams have a similar overall skill level, measured by the standard deviation of their average skill ratings.

## **Group Members**

### **Group X**

| Name                  | Student Number  |
|-----------------------|----------------|
| Philippe Dutranoit   | 20240518        |
| Name 2               | Student Number 2 |
| Name 3               | Student Number 3 |
| Name 4               | Student Number 4 |

# Imports 

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [7]:
df = pd.read_csv('../Data/players(in).csv')

In [8]:
df.drop(columns=['Unnamed: 0'], inplace=True) # drop a column that as no information 
df.head()


,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95


# Defining The constraints

In [9]:
position = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
budget = 750 
total_players = 5 # as 35 rows devided by seven position = 5 teams
team_size = sum(position.values())

we assign each player to the right position <br>
 &rarr; the dictionary contains for each position a list of the player in can be assigned

In [10]:
position_groups = {pos: df[df["Position"] == pos].values.tolist() for pos in position}

position_groups

{'GK': [['Alex Carter', 'GK', 85, 90],
  ['Jordan Smith', 'GK', 88, 100],
  ['Ryan Mitchell', 'GK', 83, 85],
  ['Chris Thompson', 'GK', 80, 80],
  ['Blake Henderson', 'GK', 87, 95]],
 'DEF': [['Daniel Foster', 'DEF', 90, 110],
  ['Lucas Bennett', 'DEF', 85, 90],
  ['Owen Parker', 'DEF', 88, 100],
  ['Ethan Howard', 'DEF', 80, 70],
  ['Mason Reed', 'DEF', 82, 75],
  ['Logan Brooks', 'DEF', 86, 95],
  ['Caleb Fisher', 'DEF', 84, 85],
  ['Maxwell Flores', 'DEF', 81, 72],
  ['Jaxon Griffin', 'DEF', 79, 65],
  ['Brayden Hughes', 'DEF', 87, 100]],
 'MID': [['Nathan Wright', 'MID', 92, 120],
  ['Connor Hayes', 'MID', 89, 105],
  ['Dylan Morgan', 'MID', 91, 115],
  ['Hunter Cooper', 'MID', 83, 85],
  ['Austin Torres', 'MID', 82, 80],
  ['Gavin Richardson', 'MID', 87, 95],
  ['Spencer Ward', 'MID', 84, 85],
  ['Bentley Rivera', 'MID', 88, 100],
  ['Dominic Bell', 'MID', 86, 95],
  ['Ashton Phillips', 'MID', 90, 110]],
 'FWD': [['Sebastian Perry', 'FWD', 95, 150],
  ['Xavier Bryant', 'FWD', 90, 

# Genetic algorytm with one constrain

to first build the algorythm we decided to make it work only with the constrain of **balance team**<br>
 &rarr; in that sence the fitness should look as shown below

In [ ]:
def fitness(individual):
    team_avgs = individual.groupby('Team')['Skill'].mean()
    return np.std(team_avgs)

In [12]:
import random
import numpy as np
import pandas as pd

# Constants
TEAM_COUNT = 5
TEAM_STRUCTURE = {
    'GK': 1,
    'DEF': 2,
    'MID': 2,
    'FWD': 2
}
POPULATION_SIZE = 100
GENERATIONS = 300
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 5

# Utility: Generate initial population
def generate_initial_population(position_groups):
    population = []
    for _ in range(POPULATION_SIZE):
        players = []
        for pos, required_per_team in TEAM_STRUCTURE.items():
            total_needed = required_per_team * TEAM_COUNT
            selected = random.sample(position_groups[pos], total_needed)
            players.extend([[p[0], p[1], p[2], p[1]] for p in selected]) # Just to be sure position is there

        random.shuffle(players)
        df = pd.DataFrame(players, columns=['Player', 'Position', 'Skill', 'Pos'])
        df['Team'] = np.repeat(range(TEAM_COUNT), sum(TEAM_STRUCTURE.values()))
        population.append(df)
    return population

# Fitness = standard deviation of average skill per team
def evaluate_fitness(individual):
    team_avgs = individual.groupby('Team')['Skill'].mean()
    return np.std(team_avgs)

# Tournament selection
def tournament_selection(population):
    selected = random.sample(population, TOURNAMENT_SIZE)
    return min(selected, key=evaluate_fitness)

# Crossover: single-point
def crossover(parent1, parent2):
    split = random.randint(1, len(parent1) - 2)
    child = pd.concat([parent1.iloc[:split], parent2.iloc[split:]]).copy()
    child = child.sample(frac=1).reset_index(drop=True)
    child['Team'] = np.repeat(range(TEAM_COUNT), sum(TEAM_STRUCTURE.values()))
    return child

# Mutation: swap same-position players
def mutate(individual):
    if random.random() < MUTATION_RATE:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        if individual.loc[idx1, 'Position'] == individual.loc[idx2, 'Position']:
            individual.loc[idx1, 'Team'], individual.loc[idx2, 'Team'] = (
                individual.loc[idx2, 'Team'],
                individual.loc[idx1, 'Team']
            )
    return individual

# Main GA function
def genetic_algorithm(position_groups):
    population = generate_initial_population(position_groups)
    best_fitness = float('inf')
    best_solution = None

    for generation in range(GENERATIONS):
        new_population = []
        for _ in range(POPULATION_SIZE):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)

        population = new_population
        best_candidate = min(population, key=evaluate_fitness)
        fitness = evaluate_fitness(best_candidate)
        if fitness < best_fitness:
            best_fitness = fitness
            best_solution = best_candidate

        if generation % 20 == 0:
            print(f"Generation {generation}: std = {fitness:.4f}")

    return best_solution

# Run GA
best_teams = genetic_algorithm(position_groups)
print(best_teams.sort_values(by='Team'))


Generation 0: std = 0.4081
Generation 20: std = 0.1400


KeyboardInterrupt: 